In [ ]:
import csv
import cv2
import numpy as np


def loadData(basePath): #new
    lines = []
    with open('data/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) ##new
        for line in reader:
            lines.append(line)
            
        for line in lines:
            source_path = line[0]
            filename = source_path.split('/')[-1]
            current_path = 'data/IMG/' + filename
            #image = cv2.imread(current_path)
            #images.append(image)
            
            
    return lines ##new

print('Data Loaded')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

def balance_data(samples, visulization_flag ,N=60, K=1,  bins=100):
    measurements = []
    for line in samples:
        measurement = float(line[3])
        measurements.append(measurement)
    n, bins, patches = plt.hist(measurements, bins=bins, color= 'orange', linewidth=0.1)
    #y_train = np.array(measurements)
    n = np.array(n)

    idx = n.argsort()[-K:][::-1]    # find the largest K bins
    del_ind = []                    # collect the index which will be removed from the data
    for i in range(K):
        if n[idx[i]] > N:
            ind = np.where((bins[idx[i]]<=measurements) & (measurements<bins[idx[i]+1]))
            ind = np.ravel(ind)
            np.random.shuffle(ind)
            del_ind.extend(ind[:len(ind)-N])

    # measurements = np.delete(measurementsangles,del_ind)
    balanced_samples = [v for i, v in enumerate(samples) if i not in del_ind]
    balanced_measurements = np.delete(measurements,del_ind)

    plt.subplot(1,2,2)
    plt.hist(balanced_measurements, bins=bins, color= 'orange', linewidth=0.1)
    plt.title('modified histogram', fontsize=20)
    plt.xlabel('steering angle', fontsize=20)
    plt.ylabel('counts', fontsize=20)

    if visulization_flag:
        plt.figure
        plt.subplot(1,2,1)
        n, bins, patches = plt.hist(measurements, bins=bins, color='orange', linewidth=0.1)
        plt.title('origin histogram', fontsize=20)
        plt.xlabel('steering angle', fontsize=20)
        plt.ylabel('counts', fontsize=20)
        plt.show()

        plt.figure
        aa = np.append(balanced_measurements, -balanced_measurements)
        bb = np.append(aa, aa)
        plt.hist(bb, bins=bins, color='orange', linewidth=0.1)
        plt.title('final histogram', fontsize=20)
        plt.xlabel('steering angle', fontsize=20)
        plt.ylabel('counts', fontsize=20)
        plt.show()

    return balanced_samples

print('Data Balanced')

In [ ]:
#PRINT HISTOGRAMS
basePath = 'data/'
print('Data loading...')
samples = loadData(basePath)
samples = balance_data(samples, visulization_flag=True)

In [ ]:
def brightness_change(image):
    """  change the brightness of the input image

    :param image: input image
    :return: new image
    """
    image_bright = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = np.random.uniform(0.2,0.8)
    image_bright[:,:,2] = image_bright[:,:,2]*random_bright
    image_bright = cv2.cvtColor(image_bright,cv2.COLOR_HSV2RGB)

    return image_bright

print('Done')

In [ ]:
def data_augmentation(images, measurements):
    """ flip every image and change the blitheness of the image, then appended to the lists

    :param images: origin image
    :param measurements: origin measurements
    :return: added augmented images and their measurements
    """
    augmented_images = []
    augmented_measurements = []
    for image, angle in zip(images, measurements):

        augmented_images.append(image)
        augmented_measurements.append(angle)

        # flip
        flipped_image = cv2.flip(image,1)
        flipped_angle = -1.0 * angle
        augmented_images.append(flipped_image)
        augmented_measurements.append(flipped_angle)

        # brightness changes
        image_b1 = brightness_change(image)
        image_b2 = brightness_change(flipped_image)

        # append images
        augmented_images.append(image_b1)
        augmented_measurements.append(angle)
        augmented_images.append(image_b2)
        augmented_measurements.append(flipped_angle)

    return augmented_images, augmented_measurements

print('Done')

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

def network_model():
    """

    :return: designed network model
    """
    model = Sequential()
    model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((70,25),(0,0))))
    model.add(Convolution2D(32,3,3,activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))
    model.add(Convolution2D(64,3,3,activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.1))
    model.add(Convolution2D(128,3,3, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Convolution2D(256,3,3, activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(20))
    model.add(Dense(1))
    return model

print('Done')

In [ ]:
def generator(samples, train_flag, batch_size=32):
    num_samples = len(samples)
    correction = 0.2  # correction angle used for the left and right images

    while 1:  # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []

            for line in batch_samples:
                angle = float(line[3])
                c_imagePath = line[0].replace(" ", "")
                filename_c = c_imagePath.split('/')[-1] #
                current_path_c = 'data/IMG/' + filename_c #                
                c_image = cv2.imread(current_path_c) #
                
                #c_image = cv2.imread(c_imagePath)
                images.append(c_image)
                angles.append(angle)
       

                if train_flag:  # only add left and right images for training data (not for validation)
                    l_imagePath = line[1].replace(" ", "")
                    filename_l = l_imagePath.split('/')[-1] #
                    current_path_l = 'data/IMG/' + filename_l #                 
                    l_image = cv2.imread(current_path_l)
                    images.append(l_image)     
                    
                    r_imagePath = line[2].replace(" ", "")
                    filename_r = r_imagePath.split('/')[-1] #
                    current_path_r = 'data/IMG/' + filename_r #                     
                    r_image = cv2.imread(current_path_r)
                    images.append(r_image)

                    angles.append(angle + correction)
                    angles.append(angle - correction)

            # flip image and change the brightness, for each input image, returns other 3 augmented images
            augmented_images, augmented_angles = data_augmentation(images, angles)

            X_train = np.array(augmented_images)
            y_train = np.array(augmented_angles)
            yield shuffle(X_train, y_train)

print('Done')

In [ ]:
# split data into training and validation
from sklearn.model_selection import train_test_split

train_samples, validation_samples = train_test_split(samples, test_size=0.3)

print('Done')

In [ ]:
# compile and train the model using the generator function
train_generator = generator(train_samples, train_flag=True, batch_size=32)
validation_generator = generator(validation_samples, train_flag=False, batch_size=32)

print('Done')

In [ ]:
# define the network model
model = network_model()
model.summary()

print('Done')

In [ ]:
nbEpoch = 4
model.compile(loss='mse', optimizer='adam')

print('Done')

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=len(train_samples)*12, epochs=nbEpoch, validation_data=validation_generator, validation_steps=len(validation_samples))

model.save('model.h5')

print('Model Saved')